In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_03 import *
import torch.nn.functional as F

In [3]:
x_train,y_train,x_valid,y_valid = get_data()
ds_train,ds_valid = Dataset(x_train, y_train),Dataset(x_valid, y_valid)

File mnist.pkl.gz already exists in data


In [4]:
nh = 50
bs = 64

In [5]:
c = y_train.max().item() + 1
loss_func = F.cross_entropy

In [6]:
class DataBunch:
    def __init__(self, dl_train, dl_valid, c):
        self.dl_train = dl_train
        self.dl_valid = dl_valid
        self.c = c
    
    @property
    def ds_train(self):
        return self.dl_train.dataset
    @property
    def ds_valid(self):
        return self.dl_valid.dataset

In [7]:
data = DataBunch(*get_dls(ds_train, ds_valid, bs), 10)

In [8]:
def get_model(data, lr=0.5, nh=50):
    n_in = data.ds_train.x.shape[1]
    model = nn.Sequential(nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, data.c))
    opt = optim.SGD(model.parameters(), lr)
    return model, opt

In [9]:
class Learner:
    def __init__(self, model, opt, loss_func, data):
        self.model = model
        self.opt = opt
        self.loss_func = loss_func
        self.data = data

In [10]:
model, opt = get_model(data)
learn = Learner(model, opt, loss_func, data)

In [11]:
from IPython.core.debugger import set_trace

In [12]:
def fit(epochs, learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb, yb in data.dl_train:
            preds = learn.model(xb)
            loss = learn.loss_func(preds, yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()
        
        learn.model.eval()
        tot_correct = tot_loss = tot_seen = 0
        with torch.no_grad():
            for xb, yb in learn.data.dl_valid:
                preds = model(xb)
                loss = learn.loss_func(preds, yb)
                tot_correct += accuracy(preds, yb).item() * len(xb)
                tot_loss += loss.item() * len(xb)
                tot_seen += len(xb)
            print(f"Acc:{tot_correct/tot_seen:<10}| Loss:{(tot_loss/tot_seen)}")

In [13]:
fit(5, learn)

Acc:0.9455    | Loss:0.1830093701004982
Acc:0.9432    | Loss:0.2086206769734621
Acc:0.9258    | Loss:0.27368702373206616
Acc:0.9135    | Loss:0.3156489304423332
Acc:0.936     | Loss:0.2227689310312271


## Callback Handler

In [14]:
# Without callbacks
def one_batch(xb,yb):
    pred = model(xb)
    loss = loss_func(pred, yb)
    loss.backward()
    opt.step()
    opt.zero_grad()

def fit():
    for epoch in range(epochs):
        for b in train_dl: one_batch(*b)

In [15]:
def onebatch(xb, yb, cbh):
    if not cbh.begin_batch(xb, yb): 
        return
    preds = cbh.learn.model(xb)
    loss = cbh.learn.loss_func(preds, yb)
    if not cbh.in_train:
        return
    if not cbh.after_loss(loss):
        return
    loss.backward()
    if cbh.after_backward():
        cbh.learn.opt.step()
    if cbh.after_step():
        cbh.learn.opt.zero_grad()
        
def all_batches(dl, cbh):
    for xb, yb in dl:
        onebatch(xb, yb, cbh)
        if cbh.do_stop():
            return

def fit(epochs, learn, cbh):
    if not cbh.begin_fit(learn): 
        return
    for epoch in range(epochs):
        if not cbh.begin_epoch(epoch):
            continue        
        all_batches(learn.data.dl_train, cbh)
        if cbh.begin_validate():
            with torch.no_grad():
                all_batches(learn.data.dl_valid, cbh)
        if cbh.do_stop() or not cbh.after_epoch():
            break
    cbh.after_fit()

In [16]:
class Callback():
    def begin_fit(self, learn):
        self.learn = learn
        return True
    def after_fit(self):
        return True
    def begin_epoch(self, epoch):
        self.epoch = epoch
        return True
    def after_epoch(self):
        return True
    def begin_validate(self):
        return True
    def begin_batch(self, xb, yb):
        self.xb = xb
        self.yb = yb
        return True
    def after_loss(self, loss):
        self.loss = loss
        return True
    def after_backward(self):
        return True
    def after_step(self):
        return True

In [20]:
class CallbackHandler():
    def __init__(self, cbs):
        self.cbs = cbs if cbs else []
    
    def begin_fit(self, learn):
        self.learn = learn
        self.in_train = True
        learn.stop = False
        res = True
        for cb in self.cbs:
            res = res and cb.begin_fit(learn)
        return res
    
    def after_fit(self):
        res = not self.in_train
        for cb in self.cbs:
            res = res and cb.after_fit()
        return res
    
    def begin_epoch(self, epoch):
        self.learn.model.train()
        self.in_train = True
        res = True
        for cb in self.cbs:
            res = res and cb.begin_epoch(epoch)
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs:
            res = res and cb.after_epoch()
        return res
    
    def begin_validate(self):
        self.learn.model.eval()
        self.in_train = False
        res = True
        for cb in self.cbs:
            res = res and cb.begin_validate()
        return res

    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs:
            res = res and cb.begin_batch(xb, yb)
        return res
    
    def after_loss(self, loss):
        res = True
        for cb in self.cbs:
            res = res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res=True
        for cb in self.cbs:
            res = res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs:
            res = res and cb.after_step()
        return res

    def do_stop(self):
        try:
            return self.learn.stop
        finally:
            self.learn.stop = False

In [21]:
class TestCallback(Callback):
    def begin_fit(self, learn):
        self.n_iter = 0
        super().begin_fit(learn)
        return True
    
    def after_step(self):
        self.n_iter += 1
        print(self.n_iter)
        if self.n_iter>=10:
            self.learn.stop = True
        return True

In [22]:
custom_callbacks = [TestCallback()]
fit(1, learn, cbh=CallbackHandler(custom_callbacks))

1
2
3
4
5
6
7
8
9
10
